In [7]:
import requests
import pyodbc
import datetime

# Open-Meteo API (No API Key Required)
API_URL = "https://api.open-meteo.com/v1/forecast?latitude=51.5074&longitude=-0.1278&current_weather=true"

# Fetch data from API
response = requests.get(API_URL)
data = response.json()

# Extract required fields
temperature = data["current_weather"]["temperature"]
windspeed = data["current_weather"]["windspeed"]
timestamp = datetime.datetime.now()

# SQL Server Connection Settings (Windows Authentication)
SERVER = "LEVENT\SQLEXPRESS"  # Replace with your SQL Server name
DATABASE = "AdventureWorks2019"

# Connect to SQL Server using Windows Authentication
conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;")
cursor = conn.cursor()

# Create Table (if it doesn't exist)
cursor.execute("""
    IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='weather_data' AND xtype='U')
    CREATE TABLE weather_data (
        id INT IDENTITY(1,1) PRIMARY KEY,
        temperature FLOAT,
        windspeed FLOAT,
        timestamp DATETIME
    )
""")
conn.commit()

# Insert data into SQL Server
cursor.execute("INSERT INTO weather_data (temperature, windspeed, timestamp) VALUES (?, ?, ?)",
               temperature, windspeed, timestamp)

# Commit transaction and close connection
conn.commit()
cursor.close()
conn.close()

print("✅ Weather data successfully inserted into SQL Server!")

✅ Weather data successfully inserted into SQL Server!


In [4]:
import requests
import pyodbc
import datetime
import time

# Open-Meteo API (No API Key Required) with Additional Fields
API_URL = "https://api.open-meteo.com/v1/forecast?latitude=51.5074&longitude=-0.1278&current_weather=true"

# SQL Server Connection Settings (Windows Authentication)
SERVER = "LEVENT\SQLEXPRESS"  # Replace with your SQL Server name
DATABASE = "AdventureWorks2019"

# Connect to SQL Server
conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;")
cursor = conn.cursor()

# Create Table (if it doesn't exist)
counter = 0

max_iterations = 5

# Run indefinitely (or set a limit)
while counter < max_iterations:
    # Fetch data from API
    response = requests.get(API_URL)
    data = response.json()

    # Extract required fields
    temperature = data["current_weather"]["temperature"]
    windspeed = data["current_weather"]["windspeed"]
    wind_gusts = data["current_weather"].get("windgusts", 0)  # Some fields may be missing
    precipitation = data["current_weather"].get("precipitation", 0)
    pressure = data["current_weather"].get("pressure", 1013)
    humidity = data["current_weather"].get("humidity", 50)
    timestamp = datetime.datetime.now()

    # Insert new row into SQL Server
    cursor.execute("INSERT INTO weather_data (temperature, windspeed, wind_gusts, precipitation, pressure, humidity, timestamp) VALUES (?, ?, ?, ?, ?, ?, ?)",
                   temperature, windspeed, wind_gusts, precipitation, pressure, humidity, timestamp)

    conn.commit()
    print(f"✅ Data inserted at {timestamp}: Temp={temperature}°C, Wind={windspeed} km/h, Gusts={wind_gusts}, Precip={precipitation}mm, Pressure={pressure}hPa, Humidity={humidity}%")

    # Wait 5 minutes before the next request
    time.sleep(3)  # 300 seconds = 5 minutes
    counter += 1


✅ Data inserted at 2025-03-04 21:40:35.815139: Temp=4.5°C, Wind=4.1 km/h, Gusts=0, Precip=0mm, Pressure=1013hPa, Humidity=50%
✅ Data inserted at 2025-03-04 21:40:39.515587: Temp=4.5°C, Wind=4.1 km/h, Gusts=0, Precip=0mm, Pressure=1013hPa, Humidity=50%
✅ Data inserted at 2025-03-04 21:40:43.203618: Temp=4.5°C, Wind=4.1 km/h, Gusts=0, Precip=0mm, Pressure=1013hPa, Humidity=50%
✅ Data inserted at 2025-03-04 21:40:46.881767: Temp=4.5°C, Wind=4.1 km/h, Gusts=0, Precip=0mm, Pressure=1013hPa, Humidity=50%
✅ Data inserted at 2025-03-04 21:40:50.558292: Temp=4.5°C, Wind=4.1 km/h, Gusts=0, Precip=0mm, Pressure=1013hPa, Humidity=50%
